#  🤖 Static Agentic RAG with CodeGen

Users can at times expect an AI application to understand their queries, even if they are vague and contain embedded semantic logic, such as a short form query that is actually compound in nature.  There is no explicit breakdown of all subqueries, because domain experts generally understand what is meant by the short form query.  In production, this is a challenging task for an AI model to handle, as foundation models are generally trained on internet scale information, which is not gauranteed to contain domain specific logic.  In fact, this kind of logic could even be harmful for generalization, as the langugage would not make much sense with respect to other contexts.  

Queries, even if clear, may be compound in nature and may require subquery decomposition for targeted QA using paradigms/techniques such as RAG.  This in turn may also necessitate the need for: reranking, vectorstore filtering for initial entity-aware document retrieval to remove/prevent noise, aggregation, etc.

Furthermore, compound queries may be comparitive in nature and may require mathematical operations, which may require the use of a code generation model to generate the code for the mathematical operations.  Alternatively, agents with tools/libraries that can handle such operations can be used, which will add more determinism, and hopefully reliability.  

Note that the longer and more complex the agentic pipeline, the more the possibilities for cumulative errors to come up, and so development and maintenance may become difficult to scale across edge cases.  If this is the case, it may be preferable to move to a dynamic, human in the loop type of system. That route has its own unique challenges, such as robust routing and team setups.  A simple example can be found here: [Corrective RAG team](./../../agent_workflows/notebooks/corrective_rag_team.ipynb).

In [ ]:
import os
import sys
import yaml
from langchain_community.vectorstores import Chroma
from IPython.display import display, Markdown

current_dir = os.getcwd()
kit_dir = os.path.abspath(os.path.join(current_dir, ".."))
repo_dir = os.path.abspath(os.path.join(kit_dir, ".."))

sys.path.append(kit_dir)
sys.path.append(repo_dir)

from utils.vectordb.vector_db import VectorDb
from utils.model_wrappers.api_gateway import APIGateway 
from utils.agents.static_RAG_with_coding import CodeRAG

examples = []

CONFIG_PATH = os.path.join(kit_dir, "config.yaml")

## Load the embedding model for semantic search

In [ ]:
def get_config_info(CONFIG_PATH: str):
        """
        Loads json config file
        """
        # Read config file
        with open(CONFIG_PATH, 'r') as yaml_file:
            config = yaml.safe_load(yaml_file)
        api_info = config["api"]
        llm_info =  config["llm"]
        embedding_model_info = config["embedding_model"]
        retrieval_info = config["retrieval"]
        prompts = config["prompts"]
        
        return api_info, llm_info, embedding_model_info, retrieval_info, prompts

def load_embedding_model(embedding_model_info: dict) -> None:
        embeddings = APIGateway.load_embedding_model(
            type=embedding_model_info["type"],
            batch_size=embedding_model_info["batch_size"],
            coe=embedding_model_info["coe"],
            select_expert=embedding_model_info["select_expert"]
            ) 
        return embeddings  

## Load the vectorstore for use as a base database

In [ ]:
_, _, embedding_model_info, _, _ = get_config_info(CONFIG_PATH=CONFIG_PATH)
embeddings = load_embedding_model(embedding_model_info=embedding_model_info)
vdb = VectorDb()
vectorstore = vdb.load_vdb(kit_dir + "/data/uber_lyft.chromadb", 
                           embeddings, 
                           collection_name='agent_workflows_default_collection')

## Load the CodeRAG static agent pipeline

We are going to answer very complex, compound questions.  Numerous components will be used for both RAG [RagComponents](./../../utils/rag/rag_components.py), which inherits methods from [BaseComponents](./../../utils/rag/base_components.py).  [CodegenComponents](./../../utils/code_gen/codegen_components.py) will also be used.  For conciseness, components will not be listed in code here, but the links should route to the respective files/modules.  

The long, CodeRAG pipeline will be explained here.  

First, we need to define the state that will be required for this large-scale question-answering task:

    class CodeRAGGraphState(TypedDict):
    """
    A typed dictionary representing the state of a CodeRAG graph.

    Args:
        question: The user's question.
        subquestions: A list of subquestions, typically generated by a LLM agent.
        entities: A list of entities determined by a LLM agent.
        generation: The most recent generation from a LLM, which may include code generation.
        documents: A list of documents that were retrieved from the vectorstore and passed retrieval grading filtering.
        answers: A list of answers that have been accumulated from the pipeline.
        original_question: The original question, which may be needed if subquestions were generated, etc.
        code: The generated code from a LLM agent.
        runnable: A binary flag of "executed" or "exception", depending on if code exeution succeeded or not.
        error: The error message from an exception if any occurred.
        rag_counter: The RAG counter.
        code_counter: The code counter, which is used for maximum amount of retries after catching exceptions.
        examples: A list of examples for query reformulation.
    """

    question : str
    subquestions: List[str]
    entities: List[str]
    generation : str
    documents : List[str]
    answers: List[str] 
    original_question: str 
    code: str
    runnable: str
    error: str
    rag_counter: int
    code_counter: int
    examples: Optional[list]

We will need state for the question, subquestions (that will be proposed by an agent), entities (that will be determined from an agent), generation (the response that is generated at various points of the pipeline), documents (the documents that are retrieved from the vectorstore), answers (the accumulated answers that are provided if subqueries were generated for answering), original_question (logging the original question in case of subquery generation and answering), code (the code that is generated by the codegen agent), runnable (the runnable code flag - executed or exception, that is generated by the agents), error (an exception message after running the Python REPL tool), rag_counter (the counter for the RAG agents), code_counter (the counter for the code generation agents), and examples (provided for the prompt reformulation, if needed).

Next, we will add the nodes for all the components we will need for th graph (excluding components to be used for conditional edges):

def create_rag_nodes(self) -> StateGraph:
        """
        Creates the nodes for the CodeRAG graph state.

        Args:
            None

        Returns:
            The StateGraph object containing the nodes for the CodeRAG graph state.
        """

        workflow: StateGraph = StateGraph(CodeRAGGraphState)

        # Define the nodes
        workflow.add_node("initialize_code_rag", self.initialize_code_rag)
        workflow.add_node("reformulate_query", self.reformulate_query)
        workflow.add_node("get_new_query", self.pass_state)
        workflow.add_node("generate_subquestions", self.generate_subquestions)
        workflow.add_node("detect_entities", self.detect_entities)
        workflow.add_node("retrieve", self.retrieve_w_filtering)
        workflow.add_node("grade_documents", self.grade_documents)
        workflow.add_node("generate", self.rag_generate)
        workflow.add_node("pass_from_qa", self.pass_state)
        workflow.add_node("pass_to_codegen", self.pass_to_codegen)
        workflow.add_node("code_generation", self.code_generation)
        workflow.add_node("determine_runnable_code", self.determine_runnable_code)
        workflow.add_node("refactor_code", self.refactor_code)
        workflow.add_node("code_error_msg", self.code_error_msg)
        workflow.add_node("failure_msg", self.failure_msg)
        workflow.add_node("aggregate_answers", self.aggregate_answers)
        workflow.add_node("return_final_answer", self.final_answer)

        return workflow


Now we will need to setup the edges and conditional edges of the graph for our apps conrtol flow:


    def build_rag_graph(self, workflow: StateGraph) -> object:
        """
        Builds a graph for the RAG workflow.

        This method constructs a workflow graph that represents the sequence of tasks
        performed by the RAG system. The graph is used to execute the workflow and
        generate code.

        Args:
            workflow: The workflow object (StateGraph containing nodes) to be modified.

        Returns:
            The compiled application object for static CodeRAG
        """

        # Build graph

        checkpointer: MemorySaver = MemorySaver()

        workflow.set_entry_point("initialize_code_rag")
        workflow.add_conditional_edges(
            "initialize_code_rag",
            self.use_examples,
            {
                "answer_generation": "get_new_query",
                "example_selection": "reformulate_query",
            },
        )
        workflow.add_edge("reformulate_query", "get_new_query")
        workflow.add_conditional_edges(
            "get_new_query",
            self.route_question,
            {
                "answer_generation": "detect_entities",
                "subquery_generation": "generate_subquestions",
            },
        )
        workflow.add_edge("generate_subquestions", "detect_entities")
        workflow.add_edge("detect_entities", "retrieve")
        workflow.add_edge("retrieve", "grade_documents")
        workflow.add_edge("grade_documents", "generate")
        workflow.add_conditional_edges(
            "generate",
            self.check_hallucinations,
            {
                "not supported": "failure_msg",
                "useful": "pass_from_qa",
                "not useful": "failure_msg",
            }
        )
        workflow.add_edge("failure_msg", "pass_from_qa")
        workflow.add_conditional_edges(
            "pass_from_qa", 
            self.determine_cont, 
            {
                "continue": "pass_to_codegen",
                "iterate": "detect_entities",
            },
        )
        workflow.add_conditional_edges(
            "pass_to_codegen",
            self.route_question_to_code,
            {
                "llm": "aggregate_answers",
                "codegen": "code_generation",
            },
        )
        workflow.add_edge("code_generation", "determine_runnable_code")
        workflow.add_conditional_edges(
        "determine_runnable_code",
        self.decide_to_refactor,
        {
            "executed": "return_final_answer", 
            "exception": "refactor_code",
            "unsuccessful": "code_error_msg"
        },
        )
        workflow.add_edge("refactor_code", "determine_runnable_code")
        workflow.add_edge("code_error_msg", "return_final_answer")
        workflow.add_edge("aggregate_answers", "return_final_answer")
        workflow.add_edge("return_final_answer", END)

        app: CompiledGraph = workflow.compile(checkpointer=checkpointer)

        return app

*Note that conditional edges come from components methods, but are not instantiated as nodes.  They are internal routers and are only used as connectors between other nodes*

In [ ]:
# add a config
config = {"configurable": {"thread_id": "1234"}}

# instantiate rag
rag = CodeRAG(
    configs=CONFIG_PATH,
    embeddings=embeddings,
    vectorstore=vectorstore,
    examples=examples,
)

# Initialize chains
rag.initialize()

# Build nodes
workflow = rag.create_rag_nodes()
print(workflow)

# Build graph
app = rag.build_rag_graph(workflow)

We have built this complex application.  Now let's visualize the graph and then have a discussion about different scenarios and how they may play out, if routed correctly via the agent system prompts.

In [ ]:
rag.display_graph(app)

## Description of pipeline

This complicated pipeline should demonstrate how seemingly straight forward user queries can require more and more complexity, which in turn means more optimization of agent system prompts, implementations for edge cases, etc.  It is recommended to start developing RAG systems for simple cases along with users and scale up complexity when really needed for user experience.  At some point, there will need to be a tradeoff between user experience/ease of use, latency (long chains will incur more and more calls to LLMs), and development challenges.

### Flow:

After initializing some counters to keep track of coding attempts, etc. The user query encounters the first conditional edge, which is an agent judge that determines, based on the [example router](./../../prompt_engineering/prompts/llama3_8b-prompt_engineering-example_judge.yaml) prompt, if the query needs reformulation or not.  If reformulation is needed, the judge determines the response to be "example selection".  Example selection routes to the reformulate query node. This node calls the reformulate_query method from the RAGComponents class.  This method calls the get_example_selector method to create a retriever for the examples.  After the retriever is instantiated, the get_examples method is called to obtain examples, based on the similarity of the example queries and the user query.  Each example contains a key value pair, which is the expected user query to be reformulated and the reformulated query.  A reformulation chain with the system prompt: [query reformulation](./../../prompt_engineering/prompts/llama3_8b-prompt_engineering-query_reformulation.yaml).  If no reformulation is needed, then the query simply passes to the next node, get_new_query, which is essentially just a pass through node that exists so we can connect a conditional edge to it (conditional edges need to be connected to nodes, not other edges).  

The next conditional edge is another agent judge.  This agent determines if the query is compound in nature and requires query decomposition, via its system prompt [subquery router](./../../prompt_engineering/prompts/llama3_8b-prompt_engineering-subquery_router.yaml).  If subquery generation is required, the query is then routed to the generate subquestions node.  This node calls the generate_subquestion method from RAGComponents, which obtains the question from the current state and passes it to the subquery_chain, which uses the prompt [subquery generation](./../../prompt_engineering/prompts/llama3_8b-prompt_engineering-subquery_generation.yaml).  Subquestions are converted to a list by splitting on new lines and then returned to subquestions in the state as an update.  If no subqueries are required, the query is simply passed along again.  

Either the question or list of subquestions are then passed to a loop, which would be flattened with the use of batch online inference, if latency requirements demand it.  This loop consists of: entity detection, retrieval with metadata filtering, document grading for relevancy, RAG generation, hallucination detection, and answer relevancy determination.  All of these steps ensure that high quality data is received and help to filter out noise when retrieving docuemtns/contexts and that answers are checked to be faithful to the source materials and helpful.  The entity detection is performed via a chain using the [entity determination](./../../prompt_engineering/prompts/llama3_8b-prompt_engineering-entity_determination.yaml) prompt and some logic that handles queries versus subqueries.  Entities are then used at the next node, retrieve, which calls the retrieve_w_filtering of RAGComponents to first filter the vectorstore by the entity name using metadata (simplified as filename, as the front ends typically involve creating vectorstores from files).  If reranking is chosen in the configs (and is recommended if not using a fine tuned embedding model), then reranking will be performed on the documents and they will be resorted.  The initial top k_retrieved_documents is the initial amount of documents to retrieve and rerank, and the final_k_retrieved_documents is the amount of documents to keep and hand to the rest of the chain.  Sequence length of the model is the main factor controlling the final k, as document grading should filter out irrelevant documents.  After documents have been obtained and passed into the graph state, they are then graded for relevancy, which is done by the grade_documents method of RAGComponents.  The grading is done by comparing the query with the document and using the retrieval_grader chain with the [retrieval grading](./../../prompt_engineering/prompts/llama3_8b-prompt_engineering-retrieval_grading.yaml) prompt.  A binary score "yes" or "no" is returned.  If the score is "yes", the document is considered relevant, if "no", it is not.  The grading is then used to filter out irrelevant documents.  Only relevant documents are then passed to the next node, RAG generation.  With the final set of documents, RAG generation is performed at the generate node, which calls the rag_generate method from the RAGComponents class.  This method formats the list of documents into a string separated by new lines and passes the formated documents to the qa_chain.  This chain takes the question (or subquestion) and the formatted documents and is instructed by the [qa](./../../prompt_engineering/prompts/llama3_8b-prompt_engineering-qa.yaml) prompt.  The current generation is updated in the state and appended to the answer list of the state.  A conditional edge is then called to check the generation for both hallucinations (the response should be grounded in the documents) and answer relevancy (the generation should actually answer the question/subquestion).  Hallucinations are checked first.  Documents are formatted, as they were in the generation step, and then passed to the hallucination chain.  The hallucination chain uses the [hallucination detection](./../../prompt_engineering/prompts/llama3_8b-prompt_engineering-hallucination_detection.yaml) prompt to provide the binary score "yes" or "no".  If scored "no", then the generation will be routed to the failure message node.  If "yes", the generation is then checked by the grading chain, which takes the question/subquestion and generation and assess relevancy based on the [retrieval grading](./../../prompt_engineering/prompts/llama3_8b-prompt_engineering-retrieval_grading.yaml) prompt.  The grade is also a binary "yes" or "no".  If "no", then the generation is also passed to the failure message.  If "yes", then the answer/generation is kept as is and the loop continues with the answer(s) appended in the state.  If the answer is deemed as not useful or if it contains hallucinations, the question is passed to the failure_chain that uses the [failure message](./../../prompt_engineering/prompts/llama3_8b-prompt_engineering-code_exec_failure.yaml) prompt to massage the message for the user, which includes providing some suggestions for finding information besides consulting the vectorstore.  

Once the question and/or all subquestions have been answered, there needs to be a determination if simple answer aggregation and summarization is required or if mathematical reasoning is needed to properly answer the question. The pass_to_codegen node collects the original question and sets it again as the question.  This is just done for simple booking keeping purposes.  A conditional edge using the route_question_to_code method from the CodeGenComponents class, is connected to this node.  At this step, the question (the original question now) is passed to the code_router chain with the [code router](./../../prompt_engineering/prompts/llama3_8b-prompt_engineering-code_router.yaml) prompt.  This chain will output either "llm" or "codegen", depending on the nature of the question.  If "llm" is determined, the aggregate answers node will call the aggregate_answers method in RAGComponents.  This method will convert the list of answers to a string seperated by two new lines so they are still somewhat individual and the question (original question) and string formatted list of answers will be passed to the aggregation_chain, which is instructed by the [answer aggregation](../../prompt_engineering/prompts/llama3_8b-prompt_engineering-answer_aggregation.yaml) prompt to answer the question as best and as fully as possible, while also allowing for partial answering over nothing.  If "codegen" is determined, the question and list of answers (formatted as a string) will be passed to the code_generation node.  The node will call the code_generation method from the CodeGenComponents class.  This method takes the questions and formatted list of answers and determines variables from those answers and tries to write code with those varables and the question.  After the code has been written, the parsed Python code snippet is passed to the determine_runnable_code node, which calls both Python REPL and the codegen_qc chain.  *It should be noted that Python REPL can execute arbitrary code on the host machine (e.g., delete files, make network requests). Use with caution.  For more information general security guidelines please see https://python.langchain.com/v0.2/docs/security/.* - From Langchain documentation @ [Python REPL tool](https://python.langchain.com/v0.2/docs/integrations/tools/python/).  If the code can run without an exception it is set as the result.  If an exception occurs, it is caught and set as the result.  Once result is populated, there is some cleanup to prevent chain related exceptions (unfriendly string characters) and it is fed into the codegen_qc chain with instructions from: [codegen_qc](./../../prompt_engineering/prompts/llama3_8b-prompt_engineering-codegen_qc.yaml).  The output of this chain is either "runnable" or "exception" and is fed to the is_runnable state.  At this point there is a conditional edge that calls decide_to_refactor from the CodeGenComponents class.  This method assesses the is_runnable output from the state and returns "executed", "exception", or "unsuccessful".  Both "executed" and "exception" are directly read from the state's last step.  However, "unsuccessful" is determined if the code_counter state variable (which is updated later) reaches the user's set limit via the configuration.  If there is an exception, the code and the exception message will be passed to the refactor_code node and method of the same name in CodeGen Components.  The code_counter will be updated by 1 and the refactor chain using the [code refactor](./../../prompt_engineering/prompts/llama3_8b-prompt_engineering-code_refactor.yaml) prompt will be invoked.  This will attempt to refactor the code, based on the model's codegen capabilities.  What follows is generally similar to the code_generation node.  Python REPL is used to try to run the refactored code.  If the code can run without an exception, the refactor_code method returns the state with updated: code, the resultant generation, and the updated code_counter.  If an exception occurs, the state is returned with updated: code, the error, and an updated code_counter.  The state is then fed back into the determine_runnable_code node to iteratively retry in a loop until either the code has been executed or the code_counter reaches the maximum number of attempts specified in the application configuration by the user.  If the code_counter exceeds the maximum number of attempts, there is routing to the code_error_msg node and method.  The final code and error/exception is then simply passed into a deterministic overall error message.  This message is appended to the answers list.  

To summarize everything that has been done and to answer the original question, the original question and final generation are passed to the final_chain with the [final chain](./../../prompt_engineering/prompts/llama3_8b-prompt_engineering-final_chain.yaml) prompt.  This massages the information into one, final answer.  If the question could not be answered, it provides some suggestionso n how a user may be able to find more information.  

### Considerations

This example should provide insight, through working examples, of how seemingly simple user expectations can balloon into very complicated implementations. Latency, cumulative error, and prompt optimization should all be considered.  It is recommended to build from simple queries to queries with a higher level of complexity and functionality demand as a project or application progresses to keep a healthy check on scaling needs for the application and to know when enough has been implemented.  In many cases, it may be that a hierarchical approach with human in the loop guidance is a better end experience than a very long and involved pipeline, even if latency is reduced via custom hardware, such as SambaNova Systems RDUs due to the other challenges mentioned.  

In [ ]:
def run_pipeline(question: str) -> None:
    
    try:
        response = rag.call_rag(app, question, config)
    except Exception as e:
        response = {}
        response["answer"] = str(e)

    display(Markdown("---Response---"))
    display(Markdown(response["answer"]))

    return response["answer"]

Let's run a number of questions of varying levels of complexity and functionality demands and assess where out of the box Llama 3 70B instruct lands for all LLM calls, without fine tuning of either the LLM or embedding model.

In [ ]:
questions = [
    "Could the trading price of lyft stock be volatile?",
    "Could the trading price of uber stock be volatile?",
    "What was the difference in revenue for Uber between 2020 and 2021?",
    "What was the difference in revenue for Lyft between 2020 and 2021?",
    "What was the change in revenue for Lyft for 2020 and 2021 as a percentage?",
    "What is 5 + 5?",
    "Provide the business overviews for Uber and Lyft.",
    "What are the growth strategies for Lyft and Uber?",
    "How do the growth strategies differ between Lyft and Uber?",
]

In [ ]:
answers = []
for question in questions:
    answers.append(run_pipeline(question))

Let's look at the question and answer pairs to get a sense of how the pipeline worked.

In [ ]:
for ques, ans in zip(questions, answers):
    display(Markdown(ques))
    display(Markdown("---ANSWER---"))
    display(Markdown(ans))

## Discussion

Many questions have been answered, but some seem to be unanswered and/or partially answered.  It generally looks like the cause is challenges with retrieval, but these questions are also naive and made without deep understanding of these documents, so the information may not always be present.  The agentic pipeline, though, seems to have all the capabilities it needs to answer these kinds of questions if information can be retrieved, from subquery generation to codegen for arithmetic reasoning.

### Questions

#### Could the trading price of lyft stock be volatile?
Information was retrieved that directly suggests that the stock may be volatile.  

#### Could the trading price of uber stock be volatile?
No explicit mention to stock volatility was found.  However, retrieved contexts contain fairly concrete information that suggests that the stock could be volatile, so the LLM was able to determine that the stock could be volatile.

#### What was the difference in revenue for Uber between 2020 and 2021?
Information for the revenue for both years was retrieved.  The original question suggested that math was required, so information was routed to codegen.  The information from the answers list was able to be parsed for inputs for the basic code that the codegen module put together.  Since this was simple code, it was executed without any errors and so refactoring loops were required.  This output was then passed to the final message node to be reformatted/stylized.

#### What was the difference in revenue for Lyft between 2020 and 2021?
Information for the revenue for both years was retrieved (it was necessary to set a high initial and final k value), incurring latency so embedding model fine tuning should be considered.  The original question suggested that math was required, so information was routed to codegen.  The information from the answers list was able to be parsed for inputs for the basic code that the codegen module, so reliabble code was generated.  The simple code was executed successfully and the output was then routed to the the final message node. 

#### What was the change in revenue for Lyft for 2020 and 2021 as a percentage?

Information for the revenue for both years was retrieved, see the note above about latency and embedding model fine tuning.  Like the question above, the question required routing to codegen.  Codegen was successful with slightly more complex logic than above.  The code executed correctly and was the output was later massaged by the final message node.

#### What is 5 + 5?

No sources could be found for this simple question.  RAG is still performed, because there is no routing, but by the time the system is routed to the code routing node, it understands that the original question requires math.  The original query is routed to the codegen node(s).  Since this is a very basic coding question, the LLM was able to create code that was executable.  The output was then passed to the final message node to be reformatted/stylized.

#### Provide the business overviews for Uber and Lyft.

All of the information for this query was contained in natural language and it could be parsed successfully.  The system was able to retrieve the information from the sources and format it in a way that was easy to understand.  The information was then passed to the final message node to be reformatted/stylized.

#### Provide the business overviews for Uber and Lyft.

Similarly to the question above, all of the information for this query was contained in natural language and it could be parsed successfully.  The system was able to retrieve the information from the sources and format it in a way that was easy to understand.  The information was then passed to the final message node to be reformatted/stylized.

#### What are the growth strategies for Lyft and Uber?

All information was successfully retrieved for this question and could be found in the plain text.  The information was then passed to the final message node to be reformatted/stylized.

#### How do the growth strategies differ between Lyft and Uber?

All information was successfully retrieved for this question and could be found in the plain text.  The information was then passed to the final message node to be reformatted/stylized.

### Improvement strategies

The pipeline seems to be able to handle all forms of queries/question provided to it, at least in terms of capability.  However, the retrieval, particularly for tabular information, remains a challenge with the OOB embedding model.  There are multiple ways to improve accuracy here for latency reduction; the embedding model could be fine tuned to improve retrieval results.  If latency is acceptable for users, the reranking model can be used with a high initial and final k for documents to avoid the embedding model fine tuning effort.